In [1]:
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
def read_data(drunkfile, normalfile):
    drunk = pd.read_csv(drunkfile)
    normal = pd.read_csv(normalfile)
    data = pd.concat([drunk, normal])
    ans_list = [1]*len(drunk) + [0]*len(normal)
    return data[['smile','anger','contempt','disgust','fear','happiness','neutral','sadness','surprise','eyeOccluded']].values, ans_list

In [3]:
X_train, Y_train = read_data("drunk_new.csv", "normal_new.csv")

In [4]:
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [5]:
X_train[X_train==True]=1
X_train[X_train==False] = 0

In [6]:
(X_train, Y_train) = shuffle(X_train, Y_train)

In [7]:
X_train, X_valid = X_train[:-50], X_train[-50:]
Y_train, Y_valid = Y_train[:-50], Y_train[-50:]

In [8]:
model = Sequential()
model.add(Dense(input_dim = X_train.shape[1], units = 10, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 8, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
callbacks = []
callbacks.append(ModelCheckpoint('models/model-{epoch:05d}-{val_acc:.5f}-{val_loss:.5f}.h5', monitor='val_acc', save_best_only=True, mode = 'auto', period=1))
model.fit(X_train, Y_train, batch_size = 8, epochs = 100, validation_split=0.2, callbacks=callbacks)

Train on 1018 samples, validate on 255 samples
Epoch 1/100
1018/1018 [==============================] - 2s - loss: 0.6409 - acc: 0.6444 - val_loss: 0.6195 - val_acc: 0.6353
Epoch 2/100
1018/1018 [==============================] - 0s - loss: 0.6286 - acc: 0.6483 - val_loss: 0.6093 - val_acc: 0.6353
Epoch 3/100
1018/1018 [==============================] - 0s - loss: 0.6093 - acc: 0.6582 - val_loss: 0.5987 - val_acc: 0.6667
Epoch 4/100
1018/1018 [==============================] - 0s - loss: 0.5998 - acc: 0.7033 - val_loss: 0.5654 - val_acc: 0.7255
Epoch 5/100
1018/1018 [==============================] - 0s - loss: 0.5603 - acc: 0.7456 - val_loss: 0.5373 - val_acc: 0.7765
Epoch 6/100
1018/1018 [==============================] - 0s - loss: 0.5502 - acc: 0.7417 - val_loss: 0.5232 - val_acc: 0.7686
Epoch 7/100
1018/1018 [==============================] - 0s - loss: 0.5390 - acc: 0.7554 - val_loss: 0.5189 - val_acc: 0.7686
Epoch 8/100
1018/1018 [==============================] - 0s - loss: 0.5

1018/1018 [==============================] - 0s - loss: 0.4553 - acc: 0.8232 - val_loss: 0.4849 - val_acc: 0.8196
Epoch 66/100
1018/1018 [==============================] - 0s - loss: 0.4544 - acc: 0.8153 - val_loss: 0.4860 - val_acc: 0.8078
Epoch 67/100
1018/1018 [==============================] - 0s - loss: 0.4636 - acc: 0.8075 - val_loss: 0.4915 - val_acc: 0.8000
Epoch 68/100
1018/1018 [==============================] - 0s - loss: 0.4586 - acc: 0.8193 - val_loss: 0.4860 - val_acc: 0.8196
Epoch 69/100
1018/1018 [==============================] - 0s - loss: 0.4524 - acc: 0.8202 - val_loss: 0.4846 - val_acc: 0.8078
Epoch 70/100
1018/1018 [==============================] - 0s - loss: 0.4511 - acc: 0.8163 - val_loss: 0.4934 - val_acc: 0.8000
Epoch 71/100
1018/1018 [==============================] - 0s - loss: 0.4605 - acc: 0.8163 - val_loss: 0.4836 - val_acc: 0.8196
Epoch 72/100
1018/1018 [==============================] - 0s - loss: 0.4577 - acc: 0.8232 - val_loss: 0.4971 - val_acc: 0.81

In [9]:
result = model.evaluate(X_valid, Y_valid, batch_size = X_valid.shape[0])
print(result[1])

50/50 [==============================] - 0s
0.860000014305


In [10]:
from xgboost import XGBClassifier
model2 = XGBClassifier(max_depth=3, n_estimators=200, learning_rate=0.05)
model2.fit(X_train, Y_train.flatten())
predict_train = model2.predict(X_train)
acc_train = np.mean(Y_train.flatten() == predict_train)
print("acc_train = ", acc_train)
predict_valid = model2.predict(X_valid)
acc_valid = np.mean(Y_valid.flatten() == predict_valid)
print("acc_valid = ", acc_valid)

acc_train =  0.859387274156
acc_valid =  0.84


c:\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
import pickle
pickle.dump(model2, open("xgbmodel", "wb"))

In [15]:
loaded_model = pickle.load(open("xgbmodel", "rb"))
acc_valid = np.mean(Y_valid.flatten() == predict_valid)
print("acc_valid = ", acc_valid)

acc_valid =  0.84


In [20]:
from keras.models import load_model
model = load_model("keras_model.h5")
result = model.evaluate(X_valid, Y_valid, batch_size = X_valid.shape[0])
print(result[1])

50/50 [==============================] - 0s
0.880000054836


In [26]:
temp = X_valid[0].reshape((1,10))
ans = model.predict(X_valid[0].reshape((1,10)))
print(ans)

[[ 0.67354691]]
